In [1]:
import sympy as sp
from sympy import Derivative as D
from utils.symbolic import (infinitesimals_generator, higher_infinitesimals_generator,
                            group_operator, der_relabel, deriv_infts, subs_new_vars, Diff)
from utils.combinatorics import list_combinatorics
from sympy.parsing.sympy_parser import parse_expr
import re
from copy import deepcopy as dc

In [2]:
x, y, z, t = sp.symbols('x y z t')
phi = sp.Function('phi')(x, y, z, t)
psi = sp.Function('psi')(x, y, z, t)
b = sp.Function('b')(x, y, z, t)
w = sp.Function('w')(x, y, z, t)

In [3]:
sigma, gamma = sp.symbols('sigma gamma')
list_indep = [x, y, z, t]
list_dep = [phi, psi, b, w]
F =  D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) + sigma*(D(psi, x)*(D(psi, x, x, y) + D(psi, y, y, y))-
        D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))) + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) - gamma*(D(b,x,x) + D(b, y, y))
F.expand()

-gamma*Derivative(b(x, y, z, t), (x, 2)) - gamma*Derivative(b(x, y, z, t), (y, 2)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (y, 3)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (x, 2), y) - sigma*Derivative(psi(x, y, z, t), (x, 3))*Derivative(psi(x, y, z, t), y) - sigma*Derivative(psi(x, y, z, t), y)*Derivative(psi(x, y, z, t), (y, 2), x) + Derivative(b(x, y, z, t), t) + Derivative(b(x, y, z, t), x)*Derivative(phi(x, y, z, t), y) - Derivative(b(x, y, z, t), y)*Derivative(phi(x, y, z, t), x) + Derivative(psi(x, y, z, t), x)*Derivative(w(x, y, z, t), y) - Derivative(psi(x, y, z, t), y)*Derivative(w(x, y, z, t), x)

In [3]:
x, t = sp.symbols('x t')
u = sp.Function('u')(x, t)
v = sp.Function('v')(x, t)

In [4]:
list_indep = [x, t]
list_dep = [u, v]
F =  D(u,t) - D(u,x,x) +D(v,t) + v
F.expand()
F_rules_array = {D(u,x,x):D(u,t)-D(v,t)-v}

In [35]:
 

infts = infinitesimals_generator(list_indep, list_dep)
n_indep = len(list_indep)
n_dep = len(list_dep)
infts_ind = infts[0:n_indep]
infts_dep = infts[n_indep:(n_dep + n_indep )]
inft_derivatives, dep_vars_derivatives = higher_infinitesimals_generator(infts_ind, infts_dep,
                                                                            2, list_indep, list_dep)
infts_dummy = [item for sublist in inft_derivatives for item in sublist]
dep_vars_derivatives = [item for sublist in dep_vars_derivatives for item in sublist]

infts = infts + infts_dummy

F, deriv_names = der_relabel(dep_vars_derivatives, F)

vars_and_derivatives = list_indep + list_dep + deriv_names

XF = group_operator(F, vars_and_derivatives, infts)

XF = subs_new_vars(dep_vars_derivatives, deriv_names, XF)

XF = XF.xreplace(F_rules_array)

# A = deriv_infts(infts_ind + infts_dep, list_indep + list_dep, 2)

# XF, deriv_infts_names = der_relabel(A, XF)

# XF = subs_new_vars(deriv_infts_names, A, XF)

# XF = subs_new_vars(deriv_names, dep_vars_derivatives, XF)

In [37]:
S = str(XF.expand())
S_no_space=S.replace(' ', '')
S_no_eta = re.sub(r'\*{2}',"&", S_no_space)

S_no_eta = re.sub(r'Subs\(Derivative\([(eta)(xi)\^][\w\(,\)\^]*\&*\d*', "", S_no_eta)
S_no_eta = re.sub(r'Derivative\([(eta)(xi)\^][\w\(,\)\^]*\&*\d*', "", S_no_eta)
S_no_eta = re.sub(r'[(eta)(xi)]+\^[\w\(,\)\^]+', "", S_no_eta)
indep_var_str = [str(ele).replace(' ', '') for ele in list_dep]
for var in indep_var_str:
    var = var.replace("(","\(").replace(")","\)")
    S_no_eta = re.sub(f'(?<!\(|,){var}', "", S_no_eta)
S_no_eta = re.sub(r'\&',"^", S_no_eta)
S_no_eta

'+*-*Derivative(u(x,t),t)*-3**Derivative(u(x,t),x)*-2**Derivative(v(x,t),x)*-2**+*Derivative(v(x,t),t)+*Derivative(v(x,t),x)+*Derivative(v(x,t),t)-*Derivative(u(x,t),x)^2+*Derivative(v(x,t),t)-*Derivative(v(x,t),(x,2))-*Derivative(v(x,t),x)^2+*Derivative(u(x,t),t)+*Derivative(v(x,t),t)+Derivative(u(x,t),t)*Derivative(u(x,t),x)^2*+2*Derivative(u(x,t),t)*Derivative(u(x,t),x)*Derivative(v(x,t),x)*+2*Derivative(u(x,t),t)*Derivative(u(x,t),x)*+2*Derivative(u(x,t),t)*Derivative(u(x,t),x)*-2*Derivative(u(x,t),t)*Derivative(v(x,t),t)*-Derivative(u(x,t),t)*Derivative(v(x,t),t)*+Derivative(u(x,t),t)*Derivative(v(x,t),x)^2*-Derivative(u(x,t),t)*Derivative(v(x,t),x)*+2*Derivative(u(x,t),t)*Derivative(v(x,t),x)*+2*Derivative(u(x,t),t)*Derivative(v(x,t),x)*+Derivative(u(x,t),t)*Derivative(v(x,t),(x,2))*+Derivative(u(x,t),t)*-Derivative(u(x,t),t)*+2*Derivative(u(x,t),t)*+Derivative(u(x,t),x)^3*+2*Derivative(u(x,t),x)^2*Derivative(v(x,t),x)*+2*Derivative(u(x,t),x)^2*-3*Derivative(u(x,t),x)*Derivative(

In [46]:
S_no = re.sub(r'\*(?=[\+\-])', "", S_no_eta)
S_no = re.sub(r'(?<!\^)\d+\*', '', S_no)
S_no = re.sub(r'(?<=[\+\-])\*', "", S_no)
S_no = re.sub(r'\*+', "*", S_no)
keys = re.split('\+|\-',S_no)

['',
 '',
 'Derivative(u(x,t),t)',
 'Derivative(u(x,t),x)',
 'Derivative(v(x,t),x)',
 '',
 'Derivative(v(x,t),t)',
 'Derivative(v(x,t),x)',
 'Derivative(v(x,t),t)',
 'Derivative(u(x,t),x)^2',
 'Derivative(v(x,t),t)',
 'Derivative(v(x,t),(x,2))',
 'Derivative(v(x,t),x)^2',
 'Derivative(u(x,t),t)',
 'Derivative(v(x,t),t)',
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)^2',
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)*Derivative(v(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),t)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),t)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)^2',
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),(x,2))',
 'Derivative(u(x,t),t)',
 'Derivative(u(x,t),t)',
 'Derivative(u(x,t),t)',
 'Derivative(u(x,t),x)^3',
 'Derivative(u(x,t),x)^2*Derivative(v(x,

In [47]:
keys_aux = []
for i in range(len(keys)):
    if keys[i]!='':
        keys_aux.append(keys[i])
keys = list(dict.fromkeys(keys_aux))

['Derivative(u(x,t),t)', 'Derivative(u(x,t),x)', 'Derivative(v(x,t),x)', 'Derivative(v(x,t),t)', 'Derivative(u(x,t),x)^2', 'Derivative(v(x,t),(x,2))', 'Derivative(v(x,t),x)^2', 'Derivative(u(x,t),t)*Derivative(u(x,t),x)^2', 'Derivative(u(x,t),t)*Derivative(u(x,t),x)*Derivative(v(x,t),x)', 'Derivative(u(x,t),t)*Derivative(u(x,t),x)', 'Derivative(u(x,t),t)*Derivative(v(x,t),t)', 'Derivative(u(x,t),t)*Derivative(v(x,t),x)^2', 'Derivative(u(x,t),t)*Derivative(v(x,t),x)', 'Derivative(u(x,t),t)*Derivative(v(x,t),(x,2))', 'Derivative(u(x,t),x)^3', 'Derivative(u(x,t),x)^2*Derivative(v(x,t),x)', 'Derivative(u(x,t),x)*Derivative(v(x,t),t)', 'Derivative(u(x,t),x)*Derivative(v(x,t),x)^2', 'Derivative(u(x,t),x)*Derivative(v(x,t),x)', 'Derivative(u(x,t),x)*Derivative(v(x,t),(x,2))', 'Derivative(u(x,t),x)*Derivative(u(x,t),t,x)', 'Derivative(v(x,t),t)^2', 'Derivative(v(x,t),t)*Derivative(v(x,t),x)', 'Derivative(v(x,t),x)*Derivative(u(x,t),t,x)', 'Derivative(u(x,t),t,x)']


In [49]:
keys_order = []
while len(keys_order) < len(keys):
    for key_1 in keys:
        inside = False
        for key_2 in keys:
            if key_2 != key_1 and (key_2 not in keys_order):
                if key_1 in key_2:
                    inside = True
                    break
        if not inside and key_1 not in keys_order:
           keys_order.append(key_1)

keys_order

['Derivative(u(x,t),t)*Derivative(u(x,t),x)^2',
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)*Derivative(v(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),t)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)^2',
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)',
 'Derivative(u(x,t),t)*Derivative(v(x,t),(x,2))',
 'Derivative(u(x,t),x)^3',
 'Derivative(u(x,t),x)^2*Derivative(v(x,t),x)',
 'Derivative(u(x,t),x)*Derivative(v(x,t),t)',
 'Derivative(u(x,t),x)*Derivative(v(x,t),x)^2',
 'Derivative(u(x,t),x)*Derivative(v(x,t),x)',
 'Derivative(u(x,t),x)*Derivative(v(x,t),(x,2))',
 'Derivative(u(x,t),x)*Derivative(u(x,t),t,x)',
 'Derivative(v(x,t),t)^2',
 'Derivative(v(x,t),t)*Derivative(v(x,t),x)',
 'Derivative(v(x,t),x)*Derivative(u(x,t),t,x)',
 'Derivative(u(x,t),t,x)',
 'Derivative(u(x,t),t)',
 'Derivative(v(x,t),t)',
 'Derivative(u(x,t),x)^2',
 'Derivative(v(x,t),(x,2))',
 'Derivative(v(x,t),x)^2',
 'Derivative(u(x,t),x)',
 'Derivative(v(x,t),x)']

In [50]:
dict_det_eqn = dict.fromkeys(keys_order)

In [51]:
dict_det_eqn

{'Derivative(u(x,t),t)*Derivative(u(x,t),x)^2': None,
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)*Derivative(v(x,t),x)': None,
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)': None,
 'Derivative(u(x,t),t)*Derivative(v(x,t),t)': None,
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)^2': None,
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)': None,
 'Derivative(u(x,t),t)*Derivative(v(x,t),(x,2))': None,
 'Derivative(u(x,t),x)^3': None,
 'Derivative(u(x,t),x)^2*Derivative(v(x,t),x)': None,
 'Derivative(u(x,t),x)*Derivative(v(x,t),t)': None,
 'Derivative(u(x,t),x)*Derivative(v(x,t),x)^2': None,
 'Derivative(u(x,t),x)*Derivative(v(x,t),x)': None,
 'Derivative(u(x,t),x)*Derivative(v(x,t),(x,2))': None,
 'Derivative(u(x,t),x)*Derivative(u(x,t),t,x)': None,
 'Derivative(v(x,t),t)^2': None,
 'Derivative(v(x,t),t)*Derivative(v(x,t),x)': None,
 'Derivative(v(x,t),x)*Derivative(u(x,t),t,x)': None,
 'Derivative(u(x,t),t,x)': None,
 'Derivative(u(x,t),t)': None,
 'Derivative(v(x,t),t)': None,
 'Derivative(u(x,t

In [52]:
S_no_space

'eta^v(x,t,u(x,t),v(x,t))+v(x,t)*Derivative(eta^u(x,t,u(x,t),v(x,t)),u(x,t))-v(x,t)*Derivative(u(x,t),t)*Derivative(xi^t(x,t,u(x,t),v(x,t)),u(x,t))-3*v(x,t)*Derivative(u(x,t),x)*Derivative(xi^x(x,t,u(x,t),v(x,t)),u(x,t))-2*v(x,t)*Derivative(v(x,t),x)*Derivative(xi^x(x,t,u(x,t),v(x,t)),v(x,t))-2*v(x,t)*Subs(Derivative(xi^x(_xi_1,t,u(x,t),v(x,t)),_xi_1),_xi_1,x)+xi^t(x,t,u(x,t),v(x,t))*Derivative(v(x,t),t)+xi^x(x,t,u(x,t),v(x,t))*Derivative(v(x,t),x)+Derivative(eta^u(x,t,u(x,t),v(x,t)),u(x,t))*Derivative(v(x,t),t)-Derivative(eta^u(x,t,u(x,t),v(x,t)),(u(x,t),2))*Derivative(u(x,t),x)**2+Derivative(eta^u(x,t,u(x,t),v(x,t)),v(x,t))*Derivative(v(x,t),t)-Derivative(eta^u(x,t,u(x,t),v(x,t)),v(x,t))*Derivative(v(x,t),(x,2))-Derivative(eta^u(x,t,u(x,t),v(x,t)),(v(x,t),2))*Derivative(v(x,t),x)**2+Derivative(eta^v(x,t,u(x,t),v(x,t)),u(x,t))*Derivative(u(x,t),t)+Derivative(eta^v(x,t,u(x,t),v(x,t)),v(x,t))*Derivative(v(x,t),t)+Derivative(u(x,t),t)*Derivative(u(x,t),x)**2*Derivative(xi^t(x,t,u(x,t),v(

In [56]:
XF_string = re.sub(r'\^',"", S_no_space)
XF_string = re.sub(r'\*{2}',"^", XF_string)
XF_string = re.sub('\+', ' +', XF_string)
XF_string = re.sub('\-', ' -', XF_string)
XF_terms = re.split(' ', XF_string)
if XF_terms[0] == '':
    XF_terms.pop(0)
XF_terms

['etav(x,t,u(x,t),v(x,t))',
 '+v(x,t)*Derivative(etau(x,t,u(x,t),v(x,t)),u(x,t))',
 '-v(x,t)*Derivative(u(x,t),t)*Derivative(xit(x,t,u(x,t),v(x,t)),u(x,t))',
 '-3*v(x,t)*Derivative(u(x,t),x)*Derivative(xix(x,t,u(x,t),v(x,t)),u(x,t))',
 '-2*v(x,t)*Derivative(v(x,t),x)*Derivative(xix(x,t,u(x,t),v(x,t)),v(x,t))',
 '-2*v(x,t)*Subs(Derivative(xix(_xi_1,t,u(x,t),v(x,t)),_xi_1),_xi_1,x)',
 '+xit(x,t,u(x,t),v(x,t))*Derivative(v(x,t),t)',
 '+xix(x,t,u(x,t),v(x,t))*Derivative(v(x,t),x)',
 '+Derivative(etau(x,t,u(x,t),v(x,t)),u(x,t))*Derivative(v(x,t),t)',
 '-Derivative(etau(x,t,u(x,t),v(x,t)),(u(x,t),2))*Derivative(u(x,t),x)^2',
 '+Derivative(etau(x,t,u(x,t),v(x,t)),v(x,t))*Derivative(v(x,t),t)',
 '-Derivative(etau(x,t,u(x,t),v(x,t)),v(x,t))*Derivative(v(x,t),(x,2))',
 '-Derivative(etau(x,t,u(x,t),v(x,t)),(v(x,t),2))*Derivative(v(x,t),x)^2',
 '+Derivative(etav(x,t,u(x,t),v(x,t)),u(x,t))*Derivative(u(x,t),t)',
 '+Derivative(etav(x,t,u(x,t),v(x,t)),v(x,t))*Derivative(v(x,t),t)',
 '+Derivative(u(x,

In [65]:
for key in dict_det_eqn.keys():
    dict_det_eqn[key] = []
lonely_terms = []
for element in XF_terms:
    add = False
    for key in dict_det_eqn:
        factors = key.split('*')
        if len(factors) == 1: 
            if key in element:
                dict_det_eqn[key].append(element.replace(key,''))
                dict_det_eqn[key][-1] = re.sub(r'(?<=[\+\-])\*+','',dict_det_eqn[key][-1])
                dict_det_eqn[key][-1] = re.sub(r'(?<=\d|\))\*+','*',dict_det_eqn[key][-1])
                if dict_det_eqn[key][-1][-1] == "*":
                    dict_det_eqn[key][-1] = dict_det_eqn[key][-1][:-1]
                add = True
                break
        else:
            inside = True
            for f in factors:
                if f not in element:
                    inside = False
                    break
            if inside:
                for f in factors:
                    element = element.replace(f, '')
                dict_det_eqn[key].append(element)
                dict_det_eqn[key][-1] = re.sub(r'(?<=[\+\-\)])\*+','',dict_det_eqn[key][-1])
                dict_det_eqn[key][-1] = re.sub(r'(?<=\d)\*+','*',dict_det_eqn[key][-1])
                add = True
                break
    if not add:
        lonely_terms.append(element)
dict_det_eqn['lonely_terms'] = lonely_terms
dict_det_eqn


{'Derivative(u(x,t),t)*Derivative(u(x,t),x)^2': ['+Derivative(xit(x,t,u(x,t),v(x,t)),(u(x,t),2))'],
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)*Derivative(v(x,t),x)': ['+2*Derivative(xit(x,t,u(x,t),v(x,t)),u(x,t),v(x,t))'],
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)': ['+2*Derivative(xix(x,t,u(x,t),v(x,t)),u(x,t))',
  '+2*Subs(Derivative(xit(_xi_1,t,u(x,t),v(x,t)),_xi_1,u(x,t)),_xi_1,x)'],
 'Derivative(u(x,t),t)*Derivative(v(x,t),t)': ['-2*Derivative(xit(x,t,u(x,t),v(x,t)),u(x,t))',
  '-Derivative(xit(x,t,u(x,t),v(x,t)),v(x,t))'],
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)^2': ['+Derivative(xit(x,t,u(x,t),v(x,t)),(v(x,t),2))'],
 'Derivative(u(x,t),t)*Derivative(v(x,t),x)': ['-Derivative(xix(x,t,u(x,t),v(x,t)),u(x,t))',
  '+2*Derivative(xix(x,t,u(x,t),v(x,t)),v(x,t))',
  '+2*Subs(Derivative(xit(_xi_1,t,u(x,t),v(x,t)),_xi_1,v(x,t)),_xi_1,x)'],
 'Derivative(u(x,t),t)*Derivative(v(x,t),(x,2))': ['+Derivative(xit(x,t,u(x,t),v(x,t)),v(x,t))'],
 'Derivative(u(x,t),x)^3': ['+Derivative(xix(x

In [26]:
dict_det_eqn['Derivative(u(x,t),t)*Derivative(u(x,t),x)']

['+2*Derivative(xi^x(x,t,u(x,t),v(x,t)),u(x,t))',
 '+2*Subs(Derivative(xi^t(_xi_1,t,u(x,t),v(x,t)),_xi_1,u(x,t)),_xi_1,x)']